# 建立自己的 package

* 這章的內容，摘要自 udemy 的課程，主要的目的包括：
  * toy example： 先快速寫一個可以 import 的 package
  * setup.py 詳細說明
  * 上傳到 pypi 的做法

* 這章的內容，會紀錄兩種打包用的 package， `distutils` (較早的工具，但 pybuilder 還是用它)，以及 `setuptools` (distutils 的強化版，現在大家主要用它)
* 會介紹 `distutils` 的原因，是因為後續會用到 `pybuilder` ，所以要先認識一下。
* 這兩款工具，都可以幫我們把現有的 package/module 打包好，給其他人使用，甚至是上傳到 Pypi 供別人下載使用。所以之後別人可以用 pip install 來安裝你的 package，並在不同專案資料夾中，直接 import 就好

## distutils

### toy example

#### 建專案資料夾與環境

* 我建立了一個專案資料夾，叫 `hylee_package`
* 並在資料夾中，先用 `python -m venv hylee_venv` 建立好虛擬環境，並使用 `source hylee_venv/bin/activate` 來進到虛擬環境 (因為我是用 mac，如果是window, 改用 `hylee_env/Scripts/activate.bat` 進到虛擬環境)
* 然後，專案資料夾內的結構如下：
* `hylee_package` (專案資料夾)
  * `hylee_env` (虛擬環境)
  * `foo.py` (module, 等等會講)
  * `setup.py` (用來打包 package 用的，等等會講)

#### 寫 module/package

* 接著，新增簡單的 module，叫做 `foo.py` ，裡面只有一個 funciton，內容如下：

In [2]:
# foo.py 的內容
def add(a, b):
    return a + b

#### 寫 setup.py

* 然後再新增一個 `setup.py` ，內容如下：

In [ ]:
from distutils.core import setup
setup(
    name='hylee_package',
    version='1.0',
    py_modules=['foo'] # module 放在哪個路徑？ 根目錄下的 foo 這裡
)

#### 執行打包命令

* 在 command line，輸入 `python setup.py sdist`
* 執行之後，專案資料夾中，多出了
  * `dist` 資料夾
    * `hylee_package-1.0.tar.gz`: 打包好的壓縮檔，就是用剛剛 setup.py 裡的 `name - version` 來命名，並打包成壓縮檔 (我是用 mac，所以壓縮成 tar.gz，如果是 windows，會壓縮成 zip)
  * `MANIFEST` 檔案
* `MANIFEST` 的內容就是純文字檔，內容如下：

In [ ]:
# file GENERATED by distutils, do NOT edit
foo.py
setup.py

* 他就是告訴我們，這個打包，總共包了這兩個檔案
* 接著，我們如果去解壓縮 `hylee_package-1.0.tar.gz` (e.g. 在 command line 打 `tar zxvf hylee_package-1.0.tar.gz`)，那解壓縮完會是一個資料夾，裡面有三個內容：
  * `foo.py` 就我的 module
  * `setup.py` 就剛剛的 setup.py
  * `PKG-INFO` 檔案，像個純文字檔，內容如下：

Metadata-Version: 1.0  
Name: hylee_package  
Version: 1.0  
Summary: UNKNOWN  
Home-page: UNKNOWN  
Author: UNKNOWN  
Author-email: UNKNOWN   
License: UNKNOWN  
Description: UNKNOWN  
Platform: UNKNOWN  

* 由此可知，他就是在介紹我們這個 package 的一些 meta information
* 很多 UNKNOWN，是因為剛剛我們在 `setup.py` 裡面都沒寫，等等更完整的例子時會補上 

#### 給別人使用

* 別人(or 自己)，可以拿到這個 `hylee_package-1.0.tar.gz` 後，進行安裝
* 有兩種方式：
  * `pip install hylee_package-1.0.tar.gz`，就直接安裝進去了
  * 先把 `hylee_package-1.0.tar.gz` 解壓縮後，進到裡面，然後下 `python setup.py install` 進行安裝

### practical example

* 有了剛剛的概念後，現在來寫一個實務上真的會用的 package

#### 建專案資料夾與環境

* 我建立了一個專案資料夾，叫 `hank_package`
* 並在資料夾中，先用 `python -m venv hk_pkg_venv` 建立好虛擬環境，並使用 `source hk_pkg_venv/bin/activate` 來進到虛擬環境 (因為我是用 mac，如果是window, 改用 `hk_pkg_venv/Scripts/activate.bat` 進到虛擬環境)
* 然後，專案資料夾內的結構如下：
* `hank_package` (專案資料夾)
  * `hk_pkg_venv` (虛擬環境)
  * `pkg1` (package, 資料夾帶有 `__init__.py` 文件)
    * `__init__.py` 
    * `sub_pkg1` (sub-package, 資料夾，裡面也帶有 `__init__.py` 文件)
      * `__init__.py`  
      * `hello.py` (module)
    * `sub_pkg2`
      * `__init__.py`
      * `greeting.py` (module)
    * `main.py` (主程式，拿 pkg1 裡面定義過的 function 做事，得到結果)
  * `pkg2` (package)
    * `__init__.py`  
    * `goodbye.py` (module)
  * `setup.py` (用來打包 package 用的，等等會講)

#### 寫 package/module

* 如上面的架構，就把 package, module 寫一寫

In [ ]:
# pkg1/sub_pkg1/hello.py
def say_hello():
    return("Hello, there!!!")

In [ ]:
# pkg1/sub_pkg2/greeting.py
def to_greet():
    return("Hi, there!!!")

In [ ]:
# pkg1/main.py
from .sub_pkg1.hello import say_hello
from .sub_pkg2.greeting import to_greet

def main():
    say_hello_content = say_hello()
    to_greet_content = to_greet()
    combine_content = f"message from main.py: {say_hello_content} & {to_greet_content}"
    print(combine_content)
    return combine_content

if __name__ == "__main__":
    main()

* 可以看到，在 `pkg1` 資料夾中，我有兩個 sub package，裡面分別各有一個 module，module底下各有 1 個 function
* 然後 `pkg1` 底下的 `main.py` 是我的主程式，內容就是會用到 pkg1 裡的 functions，然後做出結果 `combine_content`  
* 所以，本來要執行這個主程式時，會在 terminal 中下 `python -m pkg1.main` ，或是 `python pkg1/main.py`
* 最後， `pkg2` 下的 `goodbye.py` 內容如下

In [ ]:
# pkg2/goodbye.py
def say_goodbye():
    return("Goodbye my bro!!!")

#### 寫 setup.py

* 這邊要比較詳細講 `setup.py` 的內容，摘自官方文件 (https://docs.python.org/3/distutils/setupscript.html)
* `setup.py` 寫法如下：

In [ ]:
from distutils.core import setup

setup(name='hank_package',
      version='1.0',
      description='Python Distribution Utilities',
      author='Hank Lee',
      author_email='hklee@pseudo.com',
      url='https://github.com/hklee/hank_package',
      packages=[
          'pkg1', 'pkg1.sub_pkg1', 'pkg1.sub_pkg2',
          'pkg2'
      ],
      )

* 特別要注意的，就是 `packages` 的寫法。
* 在 toy_example 時，我們只有一個 module (i.e. `foo.py`)，所以，剛剛是用 `modules = ["foo"]` 來寫
* 但現在的專案，可以看到裡面是兩個 package 了 (i.e. 兩個帶有 `__init__.py` 的資料夾)，所以這邊要用 `packages = []` 來寫
* 而且，重點是，sub package 要一一標出來，如這邊的 `pkg1.sub_pkg1` 這樣。而且，`pkg1` 下面的主程式 `main.py` 不用再寫了，因為他包在 `pkg1` 下面。
* 其實，敘寫的邏輯，就是要打包的檔案的內容，例如有寫到 `pkg1` ，他就會把 pkg1 下面的檔案都抓出來 (所以包括 main.py)。但 pkg1 下面的 sub_pkg1 和 sub_pkg2 是資料夾，所以他不會抓。那我們額外又寫 `pkg1.sub_pkg1` ，就等於又指明了這個資料夾，所以他會把這個資料夾下的檔案都抓出來。最後，他就是把這些資料夾 + 檔案，幫你打包壓縮

#### 執行打包命令

* 在 command line，輸入 `python setup.py sdist`
* 執行之後，專案資料夾中，多出了
  * `dist` 資料夾
    * `hank_package-1.0.tar.gz`: 打包好的壓縮檔，就是用剛剛 setup.py 裡的 `name - version` 來命名，並打包成壓縮檔 (我是用 mac，所以壓縮成 tar.gz，如果是 windows，會壓縮成 zip)
  * `MANIFEST` 檔案
* `MANIFEST` 的內容就是純文字檔，內容如下：

In [ ]:
# file GENERATED by distutils, do NOT edit
setup.py
pkg1/__init__.py
pkg1/main.py
pkg1/sub_pkg1/__init__.py
pkg1/sub_pkg1/hello.py
pkg1/sub_pkg2/__init.py
pkg1/sub_pkg2/greeting.py
pkg2/__init__.py
pkg2/goodbye.py

* 可以看出，他就是把 setup.py ， 以及我剛剛在 setup.py 裡列出的 package 下的所有檔案，都打包
* 接著，我們如果去解壓縮 `hank_package-1.0.tar.gz` (e.g. 在 command line 打 `tar zxvf hylee_package-1.0.tar.gz`)，那解壓縮完會是一個資料夾，裡面有三個內容：
  * `pkg1` 資料夾，就是我的第一個 package
  * `pkg2` 資聊夾，就是我的第二個 package
  * `setup.py` 就剛剛的 setup.py
  * `PKG-INFO` 檔案，像個純文字檔，內容如下：

Metadata-Version: 1.0  
Name: hank_package  
Version: 1.0  
Summary: Python Distribution Utilities  
Home-page: https://github.com/hklee/hank_package  
Author: Hank Lee  
Author-email: hklee@pseudo.com  
License: UNKNOWN  
Description: UNKNOWN  
Platform: UNKNOWN  

* 由此可以知道了，剛剛在 `setup.py` 的 `packages = []` 裡面寫的，就是跟他說哪些要打包的意思 (所以，你當然也可以再寫 `modules = []`，那他也會一併打包你指定的modules)

## setuptools

* 剛剛介紹的內容，其實還缺少兩個實務上常碰到的難題：
  * 我們寫的 package，要 import 別人的套件時 (e.g. 要 import sklearn，這種 dependencies 問題)，要怎麼做？
  * 我們寫的 package，希望別人安裝後，可以直接在 terminal 下指令 (e.g. 安裝完 flask 後，在 terminal 可以直接下 flask 指令，而不用在 python 環境裡先 import flask 再下)。以剛剛的例子，我如果希望讓對方安裝完我的 package 後，只要下 `hank` 這個指令，就能執行 `pkg1/main.py` 的內容，那要怎麼做？
* 其實要新增這兩個常用功能，只要改寫剛剛的 `setup.py` 就好。在 `setup.py` 中，把剛剛 `distutil` 的指令，換成 `setuptools` 裡的指令，就可以搞定
* 所以，我們借用剛剛的 practical example 的例子，繼續下去


#### 專案資料夾, package, module 等

* 目前，專案資料夾長這樣：
* `hank_package` (專案資料夾)
  * `hk_pkg_venv` (虛擬環境)
  * `pkg1` (package, 資料夾帶有 `__init__.py` 文件)
    * `__init__.py` 
    * `sub_pkg1` (sub-package, 資料夾，裡面也帶有 `__init__.py` 文件)
      * `__init__.py`  
      * `hello.py` (module)
    * `sub_pkg2`
      * `__init__.py`
      * `greeting.py` (module)
    * `main.py` (主程式，拿 pkg1 裡面定義過的 function 做事，得到結果)
  * `pkg2` (package)
    * `__init__.py`  
    * `goodbye.py` (module)
  * `setup.py` (用來打包 package 用的，等等會講)

#### 寫 setup.py

* 然後，我們假設
  * 我們的 package 需要用到 `docutils` 和 `requests` 這兩個套件才能正常運作
  * 我希望 user 只要在 terminal 下 `hank` 這個指令，就能執行 `pkg1/main.py` 的內容
* 那作法就是，把 `setup.py` 改寫成以下的樣子：

In [ ]:
from setuptools import setup

setup(name='hank_package',
      version='1.0',
      description='Python Distribution Utilities',
      author='Hank Lee',
      author_email='hklee@pseudo.com',
      url='https://github.com/hklee/hank_package',
      packages=[
          'pkg1', 'pkg1.sub_pkg1', 'pkg1.sub_pkg2',
          'pkg2'
      ],
      install_requires=[
          'docutils>=0.3',
          'requests',
      ],
      entry_points={
          'console_scripts': [
              'hank = pkg1.main:main'
          ]
      }
      )


#### 執行打包命令

* 接著，一樣在 terminal 用 `python setup.py sdist` ，terminal 的執行過程如下：

In [ ]:
'''
(hk_pkg_venv) ➜  hank_package python setup.py sdist
running sdist
running egg_info
creating hank_package.egg-info
writing hank_package.egg-info/PKG-INFO
writing dependency_links to hank_package.egg-info/dependency_links.txt
writing entry points to hank_package.egg-info/entry_points.txt
writing requirements to hank_package.egg-info/requires.txt
writing top-level names to hank_package.egg-info/top_level.txt
writing manifest file 'hank_package.egg-info/SOURCES.txt'
reading manifest file 'hank_package.egg-info/SOURCES.txt'
writing manifest file 'hank_package.egg-info/SOURCES.txt'
warning: sdist: standard file not found: should have one of README, README.rst, README.txt, README.md

running check
creating hank_package-1.0
creating hank_package-1.0/hank_package.egg-info
creating hank_package-1.0/pkg1
creating hank_package-1.0/pkg1/sub_pkg1
creating hank_package-1.0/pkg1/sub_pkg2
creating hank_package-1.0/pkg2
copying files to hank_package-1.0...
copying setup.py -> hank_package-1.0
copying hank_package.egg-info/PKG-INFO -> hank_package-1.0/hank_package.egg-info
copying hank_package.egg-info/SOURCES.txt -> hank_package-1.0/hank_package.egg-info
copying hank_package.egg-info/dependency_links.txt -> hank_package-1.0/hank_package.egg-info
copying hank_package.egg-info/entry_points.txt -> hank_package-1.0/hank_package.egg-info
copying hank_package.egg-info/requires.txt -> hank_package-1.0/hank_package.egg-info
copying hank_package.egg-info/top_level.txt -> hank_package-1.0/hank_package.egg-info
copying pkg1/__init__.py -> hank_package-1.0/pkg1
copying pkg1/main.py -> hank_package-1.0/pkg1
copying pkg1/sub_pkg1/__init__.py -> hank_package-1.0/pkg1/sub_pkg1
copying pkg1/sub_pkg1/hello.py -> hank_package-1.0/pkg1/sub_pkg1
copying pkg1/sub_pkg2/__init__.py -> hank_package-1.0/pkg1/sub_pkg2
copying pkg1/sub_pkg2/greeting.py -> hank_package-1.0/pkg1/sub_pkg2
copying pkg2/__init__.py -> hank_package-1.0/pkg2
copying pkg2/goodbye.py -> hank_package-1.0/pkg2
Writing hank_package-1.0/setup.cfg
Creating tar archive
removing 'hank_package-1.0' (and everything under it)
'''

* 可以看到
  * 第六行，他在做 `writing dependency_links to hank_package.egg-info/dependency_links.txt`
  * 第八行他在做 `writing requirements to hank_package.egg-info/requires.txt`  
* 這都是處理 dependency 的重要文件，我們等等可以來看
* 跑完後，就 build 出 `dist` 這個資料夾，裡面就有 `hank_package-1.0.tar.gz` 這個檔。
* 我們去解壓縮這個檔，可以看到裡面有
  * `pkg1` : 和之前一樣
  * `pkg2` : 和之前一樣
  * `PKG-INFO` : 和之前一樣
  * `setup.py` : 和之前一樣
  * `setup.cfg` 新的檔案
  * `hank_package.egg-info` : 新的資料夾
    * `PKG-INFO`
    * `SOURCES.txt`
    * `dependency_links.txt`
    * `entry_points.txt`
    * `requires.txt`
    * `top_level.txt`
*  重點就在 `requires.txt` 就會列出要安裝的 package，內容就是 `docutils>=0.3 requests`
*  然後 `SOURCES.txt` 則是列出要安裝的內容有哪些

#### 安裝

* 最後，如果我有一個新的專案，要安裝這個 package，我可以這樣做
  * 開一個新專案資料夾(就叫 `hank_package_test` 好了)，在裡面建立虛擬環境 `hk_pkg_test_venv`)
  * 在這個虛擬環境中，去執行剛剛那個包好的 tar.gz 檔 `pip install hank_package-1.0.tar.gz`) 即可
* 這邊紀錄一下發現的問題：
  * 安裝完後，如果用 `pip list`，可以看到 `hank-package` 有被順利安裝
    * 但是...我原本的專案資料夾叫 `hank_package`，然後，我在 `setup.py` 裡面的 `name` 也是寫 `hank_package`，所以，我的套件名稱，應該是 `hank_package`才對啊。可能安裝時 package 名稱不能接受底線 (R其實也是)，所以他自動幫我轉成 `hank-package`
    * 但是...進到虛擬環境的 `bin/lib/python3.8/site-packages` 可以看到，他安裝的是 `hank_package-1.0-py3.8.egg-info` ，所以，搞得我有點亂。
    * 總之，這邊的 lesson learned是：專案資料夾名稱，不要有底線。然後 setup.py 的 name，也不要有底線，才能避免不一致
  * 雖然 pip list 有看到 `hank-package`，但如果你進到 python 環境，並用 `import hank-package`，他會跟你說沒有這個 package。如果改 `import hank_package`，一樣跟你說沒有這個 package
    * 這要回到虛擬環境的 `bin/lib/python3.8/site-packages` 裡面看。可以看到，他有裝成功的，是 `hank_package-1.0-py3.8.egg-info`, 和 `pkg1` 以及 `pkg2`。所以，真的有裝進去的，是 `pkg1` 和 `pkg2`!!
    * 嘗試在 python 中 import pkg1 和 pkg2 都是成功可使用的
    * 所以，這邊的 lesson learned是：
      * 專案資料夾裡面，package資料夾的名稱，才是最終的 package 要 import 的名稱，和你在 setup.py 裡的 name 無關。 setup.py 裡的 name 只是顯示在 pip list 下面而已，import 時還是看專案資料夾下的套件的資料夾名稱
      * 專案資料夾裡面，就用一個資料夾裝所有 package 就好，不要像我剛剛用了 `pkg1` 和 `pkg2`。然後，這個套件資料夾名稱，最好就是 setup.py 裡的 name，都給他用一致，才不會自己最後被搞得很混亂。
* 對於上述兩點的 lesson learned，我目前學到的 best practice 是：
  * 我如果想寫一個 `img_processing` 的 package
  * 我先建立一個專案資料夾，名稱叫 `img-processing` (不要用底線，而是一般的橫線) (rf: `scikit-learn`)
  * 在這個專案資料夾中，建立 package 資料夾，叫他 `iprocess` (重點是，不要底線也不要橫線了) (rf: `sklearn`)
  * 在這個 `iprocess` 資料夾中，可以依照功能，分很多子資料夾(sub-packages)
  * 在寫 `setup.py` 的時候， name 就直接叫 `iprocess`，也就是和package資料夾相同
  * 最後，當我們打包這個 package 給別人用的時候，對方是做 `pip install img_processing-1.0.tar.gz` 這種事 (rf: `pip install scikit-learn-1.0.tar.gz`)
  * 然後，對方要使用我們的 package 時，是用 `import iprocess` (rf: `import sklearn`)
  * 當然，我覺得如果專案資料夾名稱，直接和package名稱取完全一樣，是最不會搞亂的 (e.g. flask 的專案資料夾和package資料夾都叫 flask)
  * 但如果要讓專案資料夾比較有可讀性，可用 `-` 來處理。而重點只是，package資料夾只能一個，而且名稱要和 setup.py裡的 name 一樣，才不會搞到最後不一致，還要去 site-packages 裡面看名稱
* 希望之後能找到比上面更好的做法，例如一個專案資料夾裡面可以放多個 package 資料夾 (e.g. 我這邊的 `pkg1`, `pkg2`)，然後真的只安裝 `hank_package`，import 後，可以用 `hank_package.pkg1.xxx` 來做事。但在此之前，就先 follow 上面的 best practice 吧。